In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=0258f1184faf07573e47467467e9eecbe57ef8c1eb8edfc0466d6a553e714326
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import numpy as np


In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Spark Session

In [6]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('clustering').getOrCreate()
print('Spark Version: {}'.format(spark.version))

Spark Version: 3.4.0


In [ ]:
# spark=SparkSession.builder\
#     .master("local[*]")\
#     .appName("DescriptiveAnalysis")\
#     .config('spark.sql.execution.arrow.pyspark.enabled', True) \
#     .config('spark.sql.session.timeZone', 'UTC') \
#     .config('spark.driver.memory','32G') \
#     .config('spark.ui.showConsoleProgress', True) \
#     .config('spark.sql.repl.eagerEval.enabled', True) \
#     .getOrCreate()

# sc=spark.sparkContext

## Read Data

In [ ]:
rdd= sc.textFile('/content/drive/MyDrive/BD_Dataset')
header= rdd.first()
rdd= rdd.filter(lambda x: x!=header)
rdd.take(1)

# Clustering

In [ ]:
# select ratings and ads columns
free_apps= rdd.filter(lambda x: x.split(',')[3] == 3).count()
print(free_apps)


0


In [ ]:
ratings= rdd.map(lambda x: x.split(',')[3])
ratings.take(10)

['0.0', '4.4', '0.0', '5.0', '0.0', '0.0', '4.5', '2.0', '0.0', '4.7']

In [ ]:
ratings.count()

2312944

In [ ]:
k=4

centroids= ratings.takeSample(False, k, 0)
print(centroids)



['0.0', '0.0', '5.0', '0.0']


In [ ]:
# compute ecludian distance between x and each cetroid in centroids and return centroid with min distance
def compute_closest_centroid(x, centroids):
    distances = [ np.sqrt(np.sum((float(x) - centroid)**2)) for centroid in centroids]
    return np.argmin(distances)

# K-mean Clustring

In [ ]:
'''
map => compute distance between each point and the centroids 
produce < key
reduce => compute the new means

'''

'\nmap => compute distance between each point and the centroids \nreduce => compute the new means\n\n'

In [ ]:
centroids=[2, 4, 5, 4.5 , 0]


In [ ]:
map_result= ratings.map( lambda x:(compute_closest_centroid(x, centroids), x) )

In [ ]:
print(map_result) 
map_result.take(10)

PythonRDD[15] at RDD at PythonRDD.scala:53


[(4, '0.0'),
 (3, '4.4'),
 (4, '0.0'),
 (2, '5.0'),
 (4, '0.0'),
 (4, '0.0'),
 (3, '4.5'),
 (0, '2.0'),
 (4, '0.0'),
 (3, '4.7')]

In [ ]:
# using combine by key combine map result with key as index of centroid and value as average of all points in that centroid and number of points in that centroid
reduce_result= map_result.combineByKey(lambda value: (value, 1),
                                        lambda x, value: (float(x[0]) + value,float(x[1])  + 1),
                                        lambda x, y: (float(x[0])  +float(y[0]) , float(x[1]) + (float(y[1]) )))



In [ ]:

#End Session
spark.stop()